# Modelo 2030 - Conversión de script a Notebook
Este notebook contiene la conversión del script `scene_2030.py` a celdas Jupyter organizadas: imports, carga de datos, parámetros, definición del modelo Pyomo, resolución y resultados.

Asegúrate de tener instaladas las dependencias: `pandas`, `pyomo`, y el solver `HiGHS` (o ajustar el solver si no está disponible).

In [1]:
# Imports
import pandas as pd
import highspy as hgs
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Parámetros de configuración
tolerancia = 0.00001
perdida = 0.04

In [3]:
# Cargar archivos CSV (asegúrate que los archivos existan en el mismo directorio)
df_centrales_ex = pd.read_csv('centrales_ex.csv')
df_centrales_ex = df_centrales_ex.fillna(0)

df_centrales_nuevas = pd.read_csv('centrales_n.csv')
df_centrales_nuevas = df_centrales_nuevas.fillna(0)

# Revisar las primeras filas
#display(df_centrales_ex.head())
#display(df_centrales_nuevas.head())

In [ ]:
# Preparar parámetros (index por nombre de planta)
param_centrales = df_centrales_ex.set_index(['planta_n'])
param_centrales_nuevas = df_centrales_nuevas.set_index(['planta_n'])

# Tipos y constantes del modelo
t_centrales = ['biomasa', 'carbon','cc-gnl', 'petroleo_diesel', 'eolica','solar', 'geotermia']
t_ernc = ['eolica','solar', 'geotermia','minihidro', 'hidro_conv']
dispnibilidad_hidro = [0.8215,0.6297,0.561]
costo_falla = 505.5
year = 2030 - 2016


In [ ]:
index_plantas = param_centrales.index.tolist()
index_plantas_nuevas = param_centrales_nuevas.index.tolist()  # demanda en MW (en la pasada tenia 12k en vez de 1200 xd)
dic_bloques = {'bloque_1': {'duracion': 1200 , 'demanda' : 10233.87729},
               'bloque_2': {'duracion': 4152 , 'demanda' : 7872.0103}, 
               'bloque_3': {'duracion': 3408 , 'demanda' : 6297.872136}}

# Mostrar resumen
print('Plantillas existentes:', index_plantas[:5])
print('Plantillas nuevas:', index_plantas_nuevas[:5])

## 6. Crear Modelo y Definir Conjuntos | Parámetros | Variables

In [ ]:
# Construcción del modelo Pyomo
model = pyo.ConcreteModel()

# Conjuntos
model.CENTRALES = pyo.Set(initialize=index_plantas)
model.CENTRALES_NUEVAS = pyo.Set(initialize=index_plantas_nuevas)
model.BLOQUES = pyo.Set(initialize=['bloque_1','bloque_2','bloque_3'])

# Parámetros en formato dict (Pyomo Any para permitir dicts anidados)
model.param_centrales = pyo.Param(model.CENTRALES, initialize=param_centrales.to_dict(orient='index'), within=pyo.Any)
model.param_centrales_nuevas = pyo.Param(model.CENTRALES_NUEVAS, initialize=param_centrales_nuevas.to_dict(orient='index'), within=pyo.Any)
model.param_bloques = pyo.Param(model.BLOQUES, initialize=dic_bloques, within=pyo.Any)

# Variables(todas las generaciones son en GWh y la potencia en MW)
model.generacion_ex = pyo.Var(model.CENTRALES, model.BLOQUES, within=pyo.NonNegativeReals)
model.generacion_nuevas = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, within=pyo.NonNegativeReals)
model.potencia_in_nuevas = pyo.Var(model.CENTRALES_NUEVAS, within=pyo.NonNegativeReals)
model.falla = pyo.Var(model.BLOQUES, within=pyo.NonNegativeReals)

## 7. Definir Funciones de Restricciones

In [ ]:
# Restricciones
def fd_hidro(bloque):
    if bloque == 'bloque_1':
        return dispnibilidad_hidro[0]
    elif bloque == 'bloque_2':
        return dispnibilidad_hidro[1]
    else:
        return dispnibilidad_hidro[2]

def balance_demanda(model, bloque):
    gen_ex = sum(model.generacion_ex[planta, bloque] for planta in model.CENTRALES)
    gen_new = sum(model.generacion_nuevas[planta, bloque] for planta in model.CENTRALES_NUEVAS)
    return (gen_ex + gen_new + model.falla[bloque]) * (1000/(1+perdida)) >= model.param_bloques[bloque]['demanda'] * model.param_bloques[bloque]['duracion']
                                                    # el 1000 es para convertir de GWh a MWh

def max_gen_ex(model, planta, bloque):
    tec = model.param_centrales[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'hidro_conv', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales[planta]['disponibilidad']
        efi = model.param_centrales[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_ex[planta, bloque]*1000 <= model.param_centrales[planta]['potencia_neta_mw'] * model.param_bloques[bloque]['duracion'] * disp

def max_gen_nuevas(model, planta, bloque):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'hidro_conv', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales_nuevas[planta]['disponibilidad']
        efi = model.param_centrales_nuevas[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_nuevas[planta, bloque]*1000 <= model.potencia_in_nuevas[planta] * model.param_bloques[bloque]['duracion'] * disp

def max_capacidad_nuevas(model, planta):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_ernc:
        limite = model.param_centrales_nuevas[planta]['maxima_restriccion_2030_MW']
        return model.potencia_in_nuevas[planta] <= limite if limite > 0 else pyo.Constraint.Skip
    else:
        return pyo.Constraint.Skip

def anualidad(r, n):
    return r / (1 - (1 + r)**(-n))

# Adjuntar restricciones
model.demanda_constraint = pyo.Constraint(model.BLOQUES, rule=balance_demanda)
model.max_gen_constraint = pyo.Constraint(model.CENTRALES, model.BLOQUES, rule=max_gen_ex)
model.max_gen_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, rule=max_gen_nuevas)
model.max_capacidad_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, rule=max_capacidad_nuevas)

In [ ]:
# Función objetivo (expresiones)

# Costo operación (costos variables)
model.op_ex = pyo.Expression(expr=sum(model.generacion_ex[planta, bloque] * 1000 * # pasamos a MWh
                                (model.param_centrales[planta]['costo_variable_nc'])
                                 #+ model.param_centrales[planta]['costo_variable_t']) 
                                for planta in model.CENTRALES 
                                for bloque in model.BLOQUES))

model.op_new = pyo.Expression(expr=sum(model.generacion_nuevas[planta, bloque] * 1000 * # pasamos a MWh
                                       (model.param_centrales_nuevas[planta]['cvnc_usd_MWh'] 
                                        + model.param_centrales_nuevas[planta]['linea_peaje_usd_MWh']) 
                                for planta in model.CENTRALES_NUEVAS 
                                for bloque in model.BLOQUES))

# Costo inversión (anualidad)
model.inv_new = pyo.Expression(expr=sum(model.potencia_in_nuevas[planta]* anualidad(model.param_centrales_nuevas[planta]['tasa_descuento'], model.param_centrales_nuevas[planta]['vida_util_anos']) * model.param_centrales_nuevas[planta]['inversion_usd_kW_neto'] for planta in model.CENTRALES_NUEVAS))

# costo FALLAS (recordar que falla está en GWh)
model.costo_fallas = pyo.Expression(expr=sum(model.falla[bloque] * 1000 * # pasamos a MWh
                            costo_falla for bloque in model.BLOQUES))

r_df = 0.01
df_2016_2030 = 1 / (1 + r_df)**(year)

# minimizar (costo operacion + inversion + costo falla)
model.obj = pyo.Objective(expr = df_2016_2030 * (model.op_ex + model.op_new + model.inv_new + model.costo_fallas), sense = pyo.minimize)

In [ ]:
# Resolver el modelo
solver = pyo.SolverFactory('highs')
solver.options['mip_rel_gap'] = tolerancia
results = solver.solve(model, tee=True)
print(f"Status: {results}")

In [ ]:
# Mostrar resultados
print('\n######## Generación Existente ########')
for planta in model.CENTRALES:
    for bloque in model.BLOQUES:
        print(f'Generación de {planta} en {bloque}: {model.generacion_ex[planta, bloque].value}')

print('\n######## Generación Nuevas ########')
for planta in model.CENTRALES_NUEVAS:
    print(f'Capacidad instalada de {planta}: {model.potencia_in_nuevas[planta].value}')
    for bloque in model.BLOQUES:
        print(f'Generación de {planta} en {bloque}: {model.generacion_nuevas[planta, bloque].value}')

print('\n######## Potencia Instalada Nuevas ########')
for planta in model.CENTRALES_NUEVAS:
    print(f'Capacidad instalada de {planta}: {model.potencia_in_nuevas[planta].value}')